# EE Retrieval

In [491]:
#### For this to work you need to replace w/ your own service account and credentials
#### You may also need to install the earthengine-api
#### ! pip install earthengine-api --upgrade

# Service Account, eg "[...]iam.gserviceaccount.com"
service_account = ''
# Path to the service account key
json_credentials = ''

# Only need to initialize once in awhile
auth_intialize = False

In [492]:
import ee
import numpy as np
import os
import pandas as pd
import requests

from datetime import datetime
from PIL import Image
from tqdm import tqdm


tqdm.pandas()

In [493]:
if auth_intialize:
    SCOPES = [
        'https://www.googleapis.com/auth/earthengine',
        'https://www.googleapis.com/auth/drive'
    ]
    ee.Authenticate(scopes=SCOPES)
    credentials = ee.ServiceAccountCredentials(service_account, json_credentials)
    ee.Initialize(credentials)

## Check collections for images

In [494]:
# Example coordinates
lat, lon = 37.868710, -122.274720
point = ee.Geometry.Point([lon, lat])

# Create a buffer around the point
buffer_distance = 100
roi = point.buffer(buffer_distance)

# Define the date range of interest
start_date = '2024-01-01'
end_date = '2024-04-02'

# List of image collections to check
collections = [
    'COPERNICUS/S2_SR_HARMONIZED',
    'COPERNICUS/S1_GRD',
    'LANDSAT/LC08/C01/T1_SR',
    'LANDSAT/LE07/C01/T1_SR',
    'LANDSAT/LT05/C01/T1_SR',
    'MODIS/006/MOD09GQ',
    'MODIS/006/MOD13Q1',
    'NOAA/GOES/16/MCMIPF',
    'COPERNICUS/S5P/OFFL/L3_NO2'
]

# Check each collection for images in the specified date range
available_collections = []

for collection_id in collections:
    collection = ee.ImageCollection(collection_id).filterDate(start_date, end_date).filterBounds(roi)
    count = collection.size().getInfo()
    if count > 0:
        available_collections.append(collection_id)
        print(f"{collection_id} has {count} images available.")
    else:
        print(f"{collection_id} has no images available for the specified date range: {start_date} to {end_date}")

if not available_collections:
    print("No collections have images available for the specified date range and region.")

print("Available collections with images:", available_collections)

COPERNICUS/S2_SR_HARMONIZED has 34 images available.
COPERNICUS/S1_GRD has 23 images available.
LANDSAT/LC08/C01/T1_SR has no images available for the specified date range: 2024-01-01 to 2024-04-02
LANDSAT/LE07/C01/T1_SR has no images available for the specified date range: 2024-01-01 to 2024-04-02
LANDSAT/LT05/C01/T1_SR has no images available for the specified date range: 2024-01-01 to 2024-04-02
MODIS/006/MOD09GQ has no images available for the specified date range: 2024-01-01 to 2024-04-02
MODIS/006/MOD13Q1 has no images available for the specified date range: 2024-01-01 to 2024-04-02
NOAA/GOES/16/MCMIPF has 13203 images available.
COPERNICUS/S5P/OFFL/L3_NO2 has 1286 images available.
Available collections with images: ['COPERNICUS/S2_SR_HARMONIZED', 'COPERNICUS/S1_GRD', 'NOAA/GOES/16/MCMIPF', 'COPERNICUS/S5P/OFFL/L3_NO2']


## Retrieve images

In [495]:
def process_sentinel2(date1, date2, lon, lat, export_desc, base_dir="../Data", buffer=2560, cloud_percentage=20, cloud_percentage_fallback=50, export=False):    
    # Create point geometry
    coordinates = ee.Geometry.Point([lon, lat])
    
    # Define region of interest
    region = coordinates.buffer(buffer)

    # Load Sentinel-2 collection
    # Sentinel-2 has a max resolution of 10 m/pixel
    # Also has a 5-day revisit time
    # https://www.esa.int/Applications/Observing_the_Earth/Copernicus/Sentinel-2
    sentinel2 = ee.ImageCollection('COPERNICUS/S2_HARMONIZED') \
                    .filterBounds(region) \
                    .filterDate(date1, date2) \
                    .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', cloud_percentage))

    # If cloud_percentage threshold is too low, try again w/ a less stringent threshold
    if (sentinel2.size().getInfo() == 0):
        sentinel2 = ee.ImageCollection('COPERNICUS/S2_HARMONIZED') \
                .filterBounds(region) \
                .filterDate(date1, date2) \
                .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', cloud_percentage_fallback))

    # Create a mosaic of filtered images
    # https://developers.google.com/earth-engine/guides/ic_composite_mosaic
    mosaic = sentinel2.median()
 
    # Process and download true-color and NDVI images

    # true-color images
    # cast tc bands to the same data type
    image_uint16 = mosaic.select(['B4', 'B3', 'B2']).uint16()
    tc_params = {
        'bands': ['B4', 'B3', 'B2'],
        'min': 500,
        'max': 4000,
        'gamma': 1.4  # adjust for contrast
    }
    true_color_img = image_uint16.visualize(**tc_params)    

    # ndvi images
    ndvi_raw_img = mosaic.normalizedDifference(['B8', 'B4']).rename('NDVI')

    # ndvi two-tone color
    ndvi_color_params = {
        'min': -1,
        'max': 1,
        'palette': ['darkblue', 'white']
    }
    # ndvi_color_img = ndvi_raw_img.visualize(**ndvi_color_params)    
    
    # ndvi grayscale
    ndvi_bw_params = {
        'min': -1,
        'max': 1,
        'palette': ['black', 'white']
    }
    ndvi_bw_img = ndvi_raw_img.visualize(**ndvi_bw_params)

    image_dests = []
    image_dests.append(download_cropped_image(image=true_color_img, region=region,
                                              dirpath=f"{base_dir}/exports/tc/",
                                              export_desc=f"{export_desc}_tc"))
    # image_dests.append(download_image(ndvi_color_img, region, f"{base_dir}/exports/{export_desc}_tc_ndvi_color"))
    image_dests.append(download_cropped_image(image=ndvi_bw_img, region=region, 
                                              dirpath=f"{base_dir}/exports/ndvi/",
                                              export_desc=f"{export_desc}_ndvi_bw"))

    # Export image to Google Drive
    if export:
        raise ValueError("Export to drive should be configured separately")
        # NOTE: comment out the error above if export to drive has been configured
        export_task = ee.batch.Export.image.toDrive(
            image=image_uint16.visualize(**visualization),
            description=export_desc,
            folder='EarthEngineExports',
            scale=10,  # in meters
            region=region.bounds().getInfo()['coordinates'],
            fileFormat='GeoTIFF',
            crs='EPSG:4326',
            maxPixels=1e9
        )

        export_task.start()
    
    return image_dests

## Download and crop images to standard size

In [496]:
def find_or_create_download_dirs(base_dir):
    for dirpath in [f"{base_dir}/exports", f"{base_dir}/exports/ndvi", f"{base_dir}/exports/tc"]:
        if not os.path.exists(dirpath):
            os.makedirs(dirpath)
            print(f"Directory created at: {dirpath}")

In [497]:
def download_cropped_image(image, region, dirpath, export_desc=None, scale=10, crop_size=512):
    # Download image
    url = image.getDownloadURL({
        'scale': scale,
        'region': region.bounds().getInfo()['coordinates'],
        'format': 'GeoTIFF',
        'crs': 'EPSG:4326'
    })
    response = requests.get(url, stream=True)

    # Write to disk and crop
    if response.status_code == 200:
        export_raw_fpath = f"{dirpath}{export_desc}.tif"
        # export raw image
        with open(export_raw_fpath, 'wb') as f:
            for chunk in response.iter_content(1024):
                f.write(chunk)
        # crop image to center square
        final_image_size = crop_tif_as_center_square(export_raw_fpath)
        print(f"Image download: {export_raw_fpath} {final_image_size}")
        return export_raw_fpath
    else:
        print(f"Failed to download image. HTTP status code: {response.status_code}")
        return None

In [498]:
def crop_center(img, crop_width, crop_height):
    img_width, img_height = img.size
    return img.crop((
        (img_width - crop_width) // 2,
        (img_height - crop_height) // 2,
        (img_width + crop_width) // 2,
        (img_height + crop_height) // 2
    ))

def crop_tif_as_center_square(fpath, crop_size=512):
    img = Image.open(fpath)
    cropped_img = crop_center(img, crop_size, crop_size)
    cropped_img.save(fpath)
    return cropped_img.size

## Load wildfires dataset

In [499]:
wildfires_df = pd.read_csv("../Data/California_Fire_Incidents.csv")
wildfires_df.head(2)

,AcresBurned,Active,AdminUnit,AirTankers,ArchiveYear,CalFireIncident,CanonicalUrl,ConditionStatement,ControlStatement,Counties,...,SearchKeywords,Started,Status,StructuresDamaged,StructuresDestroyed,StructuresEvacuated,StructuresThreatened,UniqueId,Updated,WaterTenders
0,257314.0,False,Stanislaus National Forest/Yosemite National Park,NaN,2013,True,/incidents/2013/8/17/rim-fire/,NaN,NaN,Tuolumne,...,"Rim Fire, Stanislaus National Forest, Yosemite...",2013-08-17T15:25:00Z,Finalized,NaN,NaN,NaN,NaN,5fb18d4d-213f-4d83-a179-daaf11939e78,2013-09-06T18:30:00Z,NaN
1,30274.0,False,USFS Angeles National Forest/Los Angeles Count...,NaN,2013,True,/incidents/2013/5/30/powerhouse-fire/,NaN,NaN,Los Angeles,...,"Powerhouse Fire, May 2013, June 2013, Angeles ...",2013-05-30T15:28:00Z,Finalized,NaN,NaN,NaN,NaN,bf37805e-1cc2-4208-9972-753e47874c87,2013-06-08T18:30:00Z,NaN


In [500]:
wildfires_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1636 entries, 0 to 1635
Data columns (total 40 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   AcresBurned           1633 non-null   float64
 1   Active                1636 non-null   bool   
 2   AdminUnit             1636 non-null   object 
 3   AirTankers            28 non-null     float64
 4   ArchiveYear           1636 non-null   int64  
 5   CalFireIncident       1636 non-null   bool   
 6   CanonicalUrl          1636 non-null   object 
 7   ConditionStatement    284 non-null    object 
 8   ControlStatement      105 non-null    object 
 9   Counties              1636 non-null   object 
 10  CountyIds             1636 non-null   object 
 11  CrewsInvolved         171 non-null    float64
 12  Dozers                123 non-null    float64
 13  Engines               191 non-null    float64
 14  Extinguished          1577 non-null   object 
 15  Fatalities           

In [501]:
# filter records w/ missing lon/lat
missing_lonlat_df = wildfires_df[(wildfires_df["Longitude"] == 0.0) | (wildfires_df["Latitude"] == 0.0)]
missing_lonlat_df.head(5)[["Longitude", "Latitude", "Counties", "Location"]]

,Longitude,Latitude,Counties,Location
4,0.0,0.0,Ventura,Southbound Highway 101 at Camarillo Springs Ro...
27,0.0,0.0,Monterey,Pfeiffer Ridge Road and Highway 1 near Big Sur
44,0.0,0.0,Riverside,"Rio Rancho Rd & Calle Hermosa, Jurupa Valley"
48,0.0,0.0,Lake,"North of Highway 29, southeast of Kelseyville"
56,0.0,0.0,Glenn,15 miles northwest of Elk Creek off County Roa...


In [502]:
wildfires_df_cleaned = wildfires_df[~wildfires_df["UniqueId"].isin(missing_lonlat_df["UniqueId"])]
wildfires_df_cleaned

,AcresBurned,Active,AdminUnit,AirTankers,ArchiveYear,CalFireIncident,CanonicalUrl,ConditionStatement,ControlStatement,Counties,...,SearchKeywords,Started,Status,StructuresDamaged,StructuresDestroyed,StructuresEvacuated,StructuresThreatened,UniqueId,Updated,WaterTenders
0,257314.0,False,Stanislaus National Forest/Yosemite National Park,NaN,2013,True,/incidents/2013/8/17/rim-fire/,NaN,NaN,Tuolumne,...,"Rim Fire, Stanislaus National Forest, Yosemite...",2013-08-17T15:25:00Z,Finalized,NaN,NaN,NaN,NaN,5fb18d4d-213f-4d83-a179-daaf11939e78,2013-09-06T18:30:00Z,NaN
1,30274.0,False,USFS Angeles National Forest/Los Angeles Count...,NaN,2013,True,/incidents/2013/5/30/powerhouse-fire/,NaN,NaN,Los Angeles,...,"Powerhouse Fire, May 2013, June 2013, Angeles ...",2013-05-30T15:28:00Z,Finalized,NaN,NaN,NaN,NaN,bf37805e-1cc2-4208-9972-753e47874c87,2013-06-08T18:30:00Z,NaN
2,27531.0,False,CAL FIRE Riverside Unit / San Bernardino Natio...,NaN,2013,True,/incidents/2013/7/15/mountain-fire/,NaN,NaN,Riverside,...,"Mountain Fire, July 2013, Highway 243, Highway...",2013-07-15T13:43:00Z,Finalized,NaN,NaN,NaN,NaN,a3149fec-4d48-427c-8b2c-59e8b79d59db,2013-07-30T18:00:00Z,NaN
3,27440.0,False,Tahoe National Forest,NaN,2013,False,/incidents/2013/8/10/american-fire/,NaN,NaN,Placer,...,"American Fire, August 2013, Deadwood Ridge, Fo...",2013-08-10T16:30:00Z,Finalized,NaN,NaN,NaN,NaN,8213f5c7-34fa-403b-a4bc-da2ace6e6625,2013-08-30T08:00:00Z,NaN
5,22992.0,False,Sierra National Forest,NaN,2013,False,/incidents/2013/7/22/aspen-fire/,NaN,NaN,Fresno,...,"217 Aspen Fire, July 2013, Big Creek, Fresno ...",2013-07-22T22:15:00Z,Finalized,NaN,NaN,NaN,NaN,bee8c339-4f26-4b78-a5b4-a8a0ebdb8786,2013-09-24T20:15:00Z,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1631,9.0,False,CAL FIRE / Riverside County Fire,NaN,2019,True,/incidents/2019/10/10/eagle-fire/,NaN,NaN,Riverside,...,NaN,2019-10-10T12:08:00Z,Finalized,NaN,NaN,NaN,NaN,6e93b252-99a3-4214-9921-238373b17535,2019-10-10T18:11:59.733Z,NaN
1632,2.0,False,CAL FIRE Nevada-Yuba-Placer Unit,NaN,2019,True,/incidents/2019/6/28/long-fire/,NaN,NaN,Nevada,...,NaN,2019-06-28T15:03:04Z,Finalized,NaN,NaN,NaN,NaN,b38c0563-b321-431b-9174-6336c5a0d449,2019-06-30T15:52:01.023Z,NaN
1633,NaN,False,Yolo County Fire Protection District,NaN,2019,False,/incidents/2019/11/25/cashe-fire/,NaN,NaN,Yolo,...,NaN,2019-11-25T12:02:02Z,Finalized,NaN,NaN,NaN,NaN,9c26f915-1b33-422d-b30a-9eb4da6fd729,2019-12-03T16:35:20.93Z,NaN
1634,NaN,False,Camp Pendleton Marine Corps Base,NaN,2019,False,/incidents/2019/10/22/oak-fire/,NaN,NaN,San Diego,...,NaN,2019-10-22T19:20:44Z,Finalized,NaN,NaN,NaN,NaN,7264a106-e0f4-41de-8fd0-3f9110431e28,2019-11-21T12:21:28.58Z,NaN


In [503]:
wildfires_sm_df = wildfires_df_cleaned[['Longitude', 'Latitude', 'Started', 'Location', 'Counties']].copy()
wildfires_sm_df.tail()

,Longitude,Latitude,Started,Location,Counties
1631,-117.499619,33.827979,2019-10-10T12:08:00Z,"Eagle Canyon Rd. and Cajalco Rd., southwest of...",Riverside
1632,-121.000556,39.409722,2019-06-28T15:03:04Z,"Off of Long Point Road and Old Mill Road, Sou...",Nevada
1633,-121.729691,38.734634,2019-11-25T12:02:02Z,"County Road 102 and County Road 17, North of W...",Yolo
1634,-117.403719,33.351145,2019-10-22T19:20:44Z,"Near Basilone Road and Las Pulgas Road, near C...",San Diego
1635,-116.058980,33.451480,2019-10-14T15:32:20Z,Johnson Street and Ave 82,Riverside


In [504]:
# Convert started col to dt and sort by started
wildfires_sm_df['Started'] = pd.to_datetime(wildfires_sm_df['Started'], format='ISO8601')
wildfires_sm_df.sort_values(by="Started", inplace=True)

In [505]:
# Get dt one day and one day + one week before fire started
# This is to account for satellite revisit time and to avoid the date of the fire
wildfires_sm_df['Day_Before_DT'] = wildfires_sm_df['Started'] - pd.Timedelta(days=1)
wildfires_sm_df['Week_Before_DT'] = wildfires_sm_df['Day_Before_DT'] - pd.Timedelta(weeks=1)
wildfires_sm_df['TwoWeek_Before_DT'] = wildfires_sm_df['Day_Before_DT'] - pd.Timedelta(weeks=2)
wildfires_sm_df['FourWeek_Before_DT'] = wildfires_sm_df['Day_Before_DT'] - pd.Timedelta(weeks=4)

wildfires_sm_df['Day_Before'] = wildfires_sm_df['Day_Before_DT'].dt.date.astype(str)
wildfires_sm_df['Week_Before'] = wildfires_sm_df['Week_Before_DT'].dt.date.astype(str)
wildfires_sm_df['TwoWeek_Before'] = wildfires_sm_df['TwoWeek_Before_DT'].dt.date.astype(str)
wildfires_sm_df['FourWeek_Before'] = wildfires_sm_df['FourWeek_Before_DT'].dt.date.astype(str)

In [506]:
wildfires_sm_df.head()

,Longitude,Latitude,Started,Location,Counties,Day_Before_DT,Week_Before_DT,TwoWeek_Before_DT,FourWeek_Before_DT,Day_Before,Week_Before,TwoWeek_Before,FourWeek_Before
1261,-122.767510,38.071350,1969-12-31 16:00:00+00:00,"Platform Bridge Rd. and Point Reyes Station, n...",Marin,1969-12-30 16:00:00+00:00,1969-12-23 16:00:00+00:00,1969-12-16 16:00:00+00:00,1969-12-02 16:00:00+00:00,1969-12-30,1969-12-23,1969-12-16,1969-12-02
1019,-121.077610,37.218120,1969-12-31 16:00:00+00:00,"Interstate 5 near Taglio Road, Gustine",Merced,1969-12-30 16:00:00+00:00,1969-12-23 16:00:00+00:00,1969-12-16 16:00:00+00:00,1969-12-02 16:00:00+00:00,1969-12-30,1969-12-23,1969-12-16,1969-12-02
41,-118.016510,36.602575,2013-02-24 08:16:00+00:00,"south of Narrow Gauge Rd & north of Hwy 136, e...",Inyo,2013-02-23 08:16:00+00:00,2013-02-16 08:16:00+00:00,2013-02-09 08:16:00+00:00,2013-01-26 08:16:00+00:00,2013-02-23,2013-02-16,2013-02-09,2013-01-26
147,-116.941311,34.288877,2013-04-20 17:30:00+00:00,"west of Delamar Mountain, north of the communi...",San Bernardino,2013-04-19 17:30:00+00:00,2013-04-12 17:30:00+00:00,2013-04-05 17:30:00+00:00,2013-03-22 17:30:00+00:00,2013-04-19,2013-04-12,2013-04-05,2013-03-22
50,-119.635004,37.116295,2013-04-30 12:59:00+00:00,Between Road 210 and Road 200 near Fine Gold C...,Madera,2013-04-29 12:59:00+00:00,2013-04-22 12:59:00+00:00,2013-04-15 12:59:00+00:00,2013-04-01 12:59:00+00:00,2013-04-29,2013-04-22,2013-04-15,2013-04-01


In [507]:
def retrieve_set(row, base_dir="../Data"):
    try:
        process_sentinel2(date1=row['FourWeek_Before'], date2=row['Day_Before'], 
                          lon=row['Longitude'], lat=row['Latitude'], 
                          export_desc=f"{row['FourWeek_Before']}-{row['Day_Before']}-{row['Longitude']}-{row['Latitude']}",
                          base_dir=base_dir)
    except Exception as e:
        print(f"EXCEPTION - Row {row.name} @ ({row['Longitude']}, {row['Latitude']})")
        print(e)

## Small export tests

In [509]:
def example_test(fname, base_dir):
    find_or_create_download_dirs(base_dir=base_dir)

    # Define date range
    date1 = '2024-06-01'
    date2 = '2024-06-30'

    # Define coordinates

    # Test 1
    # Golden Gate Park, SF
    lon1 = -122.476944
    lat1 = 37.769722
    
    # Test 2
    # Berkeley, CA
    lon2 = -122.274720
    lat2 = 37.868710

    # Define buffer area
    # (m around lat/lon)
    buffer = 2560 

    # Define cloud percentage
    cloud_percentage = 20

    # Specify export description
    # related to export filename
    export_desc = fname

    process_sentinel2(date1, date2, lon1, lat1, export_desc + "_t1", base_dir, buffer, cloud_percentage)
    process_sentinel2(date1, date2, lon2, lat2, export_desc + "_t2", base_dir, buffer, cloud_percentage)


example_test(fname='0724__test_v16', base_dir="./DebugTest")

Directory created at: ./DebugTest/exports
Directory created at: ./DebugTest/exports/ndvi
Directory created at: ./DebugTest/exports/tc
Image download: ./DebugTest/exports/tc/0724__test_v16_t1_tc.tif (512, 512)
Image download: ./DebugTest/exports/ndvi/0724__test_v16_t1_ndvi_bw.tif (512, 512)
Image download: ./DebugTest/exports/tc/0724__test_v16_t2_tc.tif (512, 512)
Image download: ./DebugTest/exports/ndvi/0724__test_v16_t2_ndvi_bw.tif (512, 512)


In [515]:
def test_for_cloudy_row(row_idx=508, base_dir="./DebugTest"):
    find_or_create_download_dirs(base_dir=base_dir)
    df_one_row = wildfires_from_2016.loc[row_idx]
    retrieve_set(df_one_row, base_dir)

test_for_cloudy_row()

Directory created at: ./DebugTest/exports
Directory created at: ./DebugTest/exports/ndvi
Directory created at: ./DebugTest/exports/tc
Image download: ./DebugTest/exports/tc/2016-04-11-2016-05-09--120.39993-35.61961_tc.tif (512, 512)
Image download: ./DebugTest/exports/ndvi/2016-04-11-2016-05-09--120.39993-35.61961_ndvi_bw.tif (512, 512)


In [516]:
wildfires_from_2016 = wildfires_sm_df[wildfires_sm_df["Started"].dt.year >= 2016]

In [517]:
wildfires_from_2016[:5].progress_apply(lambda row: retrieve_set(row, "./DebugTest"), axis=1)

  0%|                                                                                                | 0/5 [00:00<?, ?it/s]

Image download: ./DebugTest/exports/tc/2016-03-21-2016-04-18--118.7892593-34.6888731_tc.tif (512, 512)


 40%|███████████████████████████████████▏                                                    | 2/5 [00:05<00:07,  2.51s/it]

Image download: ./DebugTest/exports/ndvi/2016-03-21-2016-04-18--118.7892593-34.6888731_ndvi_bw.tif (512, 512)
Image download: ./DebugTest/exports/tc/2016-03-26-2016-04-23--121.08036-37.2171_tc.tif (512, 512)


 60%|████████████████████████████████████████████████████▊                                   | 3/5 [00:09<00:07,  3.50s/it]

...

100%|████████████████████████████████████████████████████████████████████████████████████████| 5/5 [00:13<00:00,  2.44s/it]

EXCEPTION - Row 597 @ (-121.0381507, 35.8378273)
Image.select: Band pattern 'B4' was applied to an Image with no bands. See https://developers.google.com/earth-engine/guides/debugging#no-bands


100%|████████████████████████████████████████████████████████████████████████████████████████| 5/5 [00:14<00:00,  2.88s/it]

EXCEPTION - Row 514 @ (-119.38248, 36.78769)
Image.select: Band pattern 'B4' was applied to an Image with no bands. See https://developers.google.com/earth-engine/guides/debugging#no-bands


615    None
587    None
508    None
597    None
514    None
dtype: object

## Export all

In [508]:
base_dir = "../Data"
find_or_create_download_dirs(base_dir=base_dir)
wildfires_sm_df.progress_apply(lambda row: retrieve_set(row, base_dir), axis=1)